In [1]:
"""
Mem3dg implementations of `process_bigraph.Process()` interface
"""

'\nMem3dg implementations of `process_bigraph.Process()` interface\n'

In [2]:
from bsp import app_registrar

app_registrar.registered_addresses

['console-emitter',
 'ram-emitter',
 'composite',
 'cobra-process',
 'sed-cobra-process',
 'dynamic-fba',
 'copasi-process',
 'sed-copasi-process',
 'membrane-process',
 'simple-membrane-process',
 'smoldyn-process']

In [3]:
from process_bigraph import Composite, pp
from bsp.utils.base_utils import new_document
import numpy as np


test_state_init_fp = '../test-membrane-initial.json'
oblate = "../fixtures/sample_meshes/oblate.ply"
prolate = "../fixtures/sample_meshes/prolate.ply"

membrane_config = {
    # 'mesh_file': oblate,
    'geometry': {
        'type': 'icosphere',
        'params': {
            'radius': 1.0,
            'subdivision': 2
        }
    },
    'tension_model': {
        'modulus': 0.1, 
        'preferredArea': 12.4866
    },
    'osmotic_model': {
        'preferredVolume': 0.7 * np.pi * 4 / 3,
        'reservoirVolume': 0,
        'strength': 0.02
    },
    'parameters': {
        'bending': {
            'Kbc': 8.22e-5
        }
    },
    'save_period': 100,
    'tolerance': 1e-11,
    'characteristic_time_step': 2
}

membrane_port_stores = {
    'geometry': ['geometry_store'],
    "velocities": ["velocities_store"]
    # "parameters": ["parameters_store"],
}

state = new_document(
    name='membrane', 
    address='simple-membrane-process',
    _type='process', 
    config=membrane_config, 
    inputs=membrane_port_stores,
    outputs=membrane_port_stores,
    add_emitter=False
)

state['emitter'] = {
    'address': 'local:ram-emitter',
    '_type': 'step',
    'config': {
        'emit': {
            # "geometry": {
            #     "faces": "list[list[integer]]",
            #     "vertices": "list[list[float]]",
            # },
            'velocities': 'list[list[float]]',
            "geometry": 'tree',
            # "parameters": "tree[float]",
            # "velocities": "list[list[float]]"
        }
    },
    'inputs': { 
        'geometry': ['geometry_store'],
        'velocities': ['velocities_store']
    }
}
        
simulation = Composite(
    config={'state': state},
    core=app_registrar.core
)

pp(state)

{ 'emitter': { '_type': 'step',
               'address': 'local:ram-emitter',
               'config': { 'emit': { 'geometry': 'tree',
                                     'velocities': 'list[list[float]]'}},
               'inputs': { 'geometry': ['geometry_store'],
                           'velocities': ['velocities_store']}},
  'global_time': 0.0,
  'membrane': { '_type': 'process',
                'address': 'local:simple-membrane-process',
                'config': { 'characteristic_time_step': 2,
                            'geometry': { 'params': { 'radius': 1.0,
                                                      'subdivision': 2},
                                          'type': 'icosphere'},
                            'osmotic_model': { 'preferredVolume': 2.9321531433504737,
                                               'reservoirVolume': 0,
                                               'strength': 0.02},
                            'parameters': {'bending': {'Kbc': 

In [4]:
# define simulation.run param, aka: total_time in mem3dg?

simulation.run(100)

Initialized NetCDF file at /var/folders/yy/8crj8x7x5_3b86f0js6_0bn00000gr/T/tmpnu_4h7_z/traj.nc

t: 0, n: 0, isSmooth: 1
A, tension: 12.4783, -6.67879e-05, V, pressure: 4.02477, -0.00254171, h: 0.80235
nFaces: 892
minE: 0.14732; maxE: 0.217463
minA: 0.0124267; maxA: 0.0156246
E_total: 0.0035835
E_kin: 0
E_pot: 0.0035835
W_ext: 0
|e|Mech: 0.0015849
|e|Chem: 0
H: [0.61458,1.48814]
K: [0.411477,1.78376]
phi: [1,1]
sum_phi: 12.4783
H0: [0,0]

Reached time.

t: 2, n: 1, isSmooth: 1
A, tension: 12.4749, -9.34783e-05, V, pressure: 4.0232, -0.00253804, h: 0.802251
nFaces: 892
minE: 0.147305; maxE: 0.217456
minA: 0.0124241; maxA: 0.0156223
E_total: 0.00357987
E_kin: 1.25595e-06
E_pot: 0.00357862
W_ext: 0
|e|Mech: 2.5119e-06
|e|Chem: 0
H: [0.617808,1.4685]
K: [0.415781,1.73412]
phi: [1,1]
sum_phi: 12.4749
H0: [0,0]
Simulation finished, and data saved to /var/folders/yy/8crj8x7x5_3b86f0js6_0bn00000gr/T/tmpnu_4h7_z
Closed NetCDF file


TypeError: __init__(): incompatible constructor arguments. The following argument types are supported:
    1. pymem3dg._core.Geometry(inputMesh: str, notableVertex: numpy.ndarray[bool[m, 1]])
    2. pymem3dg._core.Geometry(inputMesh: str)
    3. pymem3dg._core.Geometry(faceMatrix: numpy.ndarray[numpy.uint64[m, 3]], vertexMatrix: numpy.ndarray[numpy.float64[m, 3]], notableVertex: numpy.ndarray[bool[m, 1]])
    4. pymem3dg._core.Geometry(faceMatrix: numpy.ndarray[numpy.uint64[m, 3]], vertexMatrix: numpy.ndarray[numpy.float64[m, 3]])
    5. pymem3dg._core.Geometry(trajFile: str, startingFrame: int)

Invoked with: array([[[  2,   1,   0],
        [  4,   2,   0],
        [  4,   0,   3],
        ...,
        [447, 421, 419],
        [447, 419, 446],
        [447, 413, 421]],

       [[  2,   1,   0],
        [  4,   2,   0],
        [  4,   0,   3],
        ...,
        [447, 421, 419],
        [447, 419, 446],
        [447, 413, 421]]], dtype=uint32), array([[[-0.062784  , -0.415658  , -0

In [5]:
results = simulation.gather_results()

In [6]:
results

{('emitter',): [{'geometry': {'faces': [[2, 1, 0],
     [4, 2, 0],
     [4, 0, 3],
     [5, 4, 11],
     [8, 7, 6],
     [0, 8, 9],
     [1, 8, 0],
     [8, 6, 9],
     [10, 3, 0],
     [10, 0, 9],
     [11, 4, 3],
     [11, 3, 12],
     [12, 3, 10],
     [13, 11, 12],
     [16, 15, 14],
     [7, 16, 6],
     [16, 14, 6],
     [6, 14, 17],
     [18, 9, 17],
     [9, 6, 17],
     [18, 10, 9],
     [19, 10, 18],
     [19, 12, 10],
     [20, 12, 19],
     [22, 13, 20],
     [21, 13, 22],
     [13, 12, 20],
     [23, 21, 22],
     [25, 24, 14],
     [25, 14, 15],
     [26, 17, 14],
     [26, 14, 24],
     [26, 18, 17],
     [27, 18, 26],
     [28, 19, 18],
     [28, 18, 27],
     [29, 20, 19],
     [29, 19, 28],
     [30, 22, 20],
     [30, 20, 29],
     [31, 23, 22],
     [31, 22, 30],
     [25, 32, 24],
     [33, 26, 24],
     [33, 24, 34],
     [34, 24, 32],
     [33, 27, 26],
     [35, 27, 33],
     [36, 28, 27],
     [36, 27, 35],
     [37, 29, 28],
     [37, 28, 36],
     [38, 30, 39

In [ ]:
test_state_init_fp = '../test-membrane-initial.json'
oblate = "../fixtures/sample_meshes/oblate.ply"
prolate = "../fixtures/sample_meshes/prolate.ply"

membrane_config = {
    'mesh_file': oblate,
    'tension_model': {
        'modulus': 0.1, 
        'preferredArea': 12.4866
    },
    'osmotic_model': {
        'preferredVolume': 0.7 * 3.14 * 4 / 3,
        'reservoirVolume': 0,
        'strength': 0.02
    },
    'parameters': {
        'bending': {
            'Kbc': 8.22e-5
        }
    },
    'save_period': 100,
    'tolerance': 1e-11,
    'characteristic_time_step': 2
}

In [ ]:
p = dg.Parameters()

p.bending.Kbc = 8.22e-5

p.tension.form = partial(
    dgb.preferredAreaSurfaceTensionModel, modulus=0.1, preferredArea=12.4866
)
p.osmotic.form = partial(
    dgb.preferredVolumeOsmoticPressureModel,
    preferredVolume=0.7 * 3.14 * 4 / 3,
    reservoirVolume=0,
    strength=0.02,
)

In [ ]:
geo = dg.Geometry(oblate)

outputDir = Path("output/tutorial1/biconcave")
outputDir.mkdir(parents=True, exist_ok=True)

g = dg.System(geometry=geo, parameters = p)
g.initialize()

fe = dg.Euler(system = g, characteristicTimeStep = 2,
              totalTime = 10000, savePeriod = 100,
              tolerance = 1e-11, outputDirectory = str(outputDir))
# Uncomment to print intermediate statistics
fe.ifPrintToConsole = True
fe.ifOutputTrajFile = True
success = fe.integrate()


In [ ]:
# plot the netcdf data in outputDir/"traj.nc" in matplotlib
import pymem3dg.visual as dg_vis
import matplotlib.pyplot as plt

from netCDF4 import Dataset

data = Dataset(str(outputDir / "traj.nc"),'r')
# get data parameters
data.groups['Trajectory'].variables

# get the coordinates from groups/trajectory.variables
x = data.groups['Trajectory'].variables['coordinates'][:]

# plot x[] in 3d spaces
# where the array is xyz, xyz, xyz

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(x[0][::3], x[0][1::3], x[0][2::3])
ax.scatter(x[-1][::3], x[-1][1::3], x[-1][2::3])

plt.show()